# Problema 1

Implementar los siguientes métodos de descenso gradiente (naïve = tamaño de paso $\alpha$ constante):

- Descenso gradiente naïve con dirección de descenso aleatoria
- Descenso máximo naïve
- Descenso gradiente de Newton, con Hessiano exacto
- Un método de gradiente conjugado (Fletcher-Reeves, Hestenes-Stiefel, Polak-Ribière)
- Método BFGS

En cada uno de los métodos, su función debe recibir los siguientes argumentos:

- La función objetivo $f$.
- El gradiente de la función objetivo $df$.
- El hessiano $ddf$ (cuando sea necesario).
- Un punto inicial $x_0 \in \mathbb{R}^n$.
- El tamaño de paso $\alpha > 0$.
- El número máximo de iteraciones $maxIter$.
- La tolerancia $\varepsilon$, así como un criterio de paro.

Como resultado, sus algoritmos deben devolver: la mejor solución encontrada *best* (la última de las aproximaciones calculadas); la secuencia de iteraciones $x_k$; la secuencia de valores $f(x_k)$; la secuencia de errores en cada paso (según el error de su criterio de paro).

Además, es deseable indicar el número de iteraciones efectuadas por el algoritmo, y si se obtuvo o no convergencia del método.

In [ ]:
import numpy as np, time

In [ ]:
SEED = 22801

> **Nota:** se puede revisar la [documentación](./reporte1.md) para saber los parámetros, retornos, funcionalidad y significado de los algoritmos.

In [ ]:
def norm(x, ord=None):
    if ord is None:
        ord = 2
    return np.linalg.norm(x, ord=ord)

In [ ]:
def projOrth(u, b_orth):
    v = u.copy().astype(float)
    if b_orth.ndim == 1:
        b = b_orth / (norm(b_orth) + 1e-15)
        v -= (v @ b) * b
    else:
        for b in b_orth:
            b = b / (norm(b) + 1e-15)
            v -= (v @ b) * b
    n = norm(v)
    if n < 1e-15:
        return v
    return v / n

## Descenso gradiente naïve con dirección de descenso aleatoria y Descenso máximo naïve

In [ ]:
def gradientDescentNaive(
    f, df, x0, alpha=0.1, maxIter=500, tol=1e-4, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=SEED, verbose=False, extra=None
):
    """
    GD naïve con dirección: aleatoria o fija (controlada por extra).
    Retorna (best, xs, fxs, errors, metrics).
    extra:
      - phiMode: "random" | "fixed"
      - phi:     float (rad) si phiMode=="fixed"
      - phiRange:(lo, hi) rad para muestreo si "random" (por defecto (-π/2, π/2))
    """
    # ====== configurar modo/ángulos (merge seguro del extra) ======
    base_extra = {"phiMode": "random", "phi": 0.0, "phiRange": (-np.pi/2, np.pi/2)}
    if extra is not None:
        base_extra.update(extra)  # el 'extra' del llamador sobreescribe
    phiMode  = base_extra["phiMode"]
    phiFixed = float(base_extra["phi"])
    lo, hi   = base_extra["phiRange"]

    rng = np.random.default_rng(randomState)

    x = np.array(x0, dtype=float).reshape(-1)
    n = x.size

    xs, fxs, errors = [x.copy()], [f(x)], []
    gradNorms, stepNorms, approxErrors, angles, dirs, xs2D, ks = [], [], [], [], [], [], []
    t0 = time.time()

    g = df(x)
    gradNorms.append(norm(g, normOrder))

    converged, stopReason = False, "maxIter"

    if verbose:
        print(f"[k=0] fx={fxs[-1]:.6e} | ||grad||={gradNorms[-1]:.6e} | x={x} | mode={phiMode}")

    for k in range(1, maxIter+1):
        # --- Selección de ángulo ---
        if phiMode == "fixed":
            phi = phiFixed
        else:
            phi = rng.uniform(lo, hi)

        # --- Dirección de descenso ---
        gnorm = norm(g, normOrder)
        if gnorm < 1e-15:
            d = -g
            angles.append(0.0)
        else:
            ghat = g / gnorm
            z = rng.normal(size=n)
            v = projOrth(z, ghat)
            if norm(v) < 1e-15:
                d = -ghat
                phi = 0.0
            else:
                d = np.cos(phi)*(-ghat) + np.sin(phi)*v

        # --- Paso naïve ---
        x_new = x + alpha * d
        fx_new = f(x_new)
        step = x_new - x

        # --- Error según stopCrit ---
        if stopCrit == "grad":
            err = norm(df(x_new), normOrder)
        elif stopCrit == "fx":
            err = abs(fx_new - fxs[-1])
        elif stopCrit == "xAbs":
            err = norm(step, normOrder)
        elif stopCrit == "xRel":
            err = norm(step, normOrder) / max(1.0, norm(x_new, normOrder))
        else:
            raise ValueError("stopCrit inválido (usa 'grad','fx','xAbs','xRel').")

        # --- Registro ---
        xs.append(x_new.copy()); fxs.append(fx_new); errors.append(err)
        gradNorms.append(norm(df(x_new), normOrder))
        stepNorms.append(norm(step, normOrder))
        approxErrors.append(err); angles.append(float(phi)); dirs.append(d.copy()); ks.append(k)
        if isPlottable and n == 2:
            xs2D.append(x_new.copy())

        if verbose:
            print(f"[k={k}] fx={fx_new:.6e} | err({stopCrit})={err:.6e} | "
                  f"||grad||={gradNorms[-1]:.6e} | ||step||={stepNorms[-1]:.6e} | phi={phi:.3f}")

        # --- Paro por tolerancia ---
        if err <= tol:
            converged, stopReason = True, "tolerance"
            x = x_new
            break

        # --- Avanzar ---
        x, g = x_new, df(x_new)

    timeSec = time.time() - t0
    best = x
    kstar = len(xs) - 1

    if verbose:
        print(f"==> stopReason={('tolerance' if converged else 'maxIter')} | "
              f"iters={kstar} | fx*={fxs[-1]:.6e} | ||grad*||={norm(df(best), normOrder):.6e}")

    # Etiqueta de método según modo
    method_label = ("Steepest Descent (naive)" if (phiMode == "fixed" and abs(phiFixed) < 1e-14)
                    else "Gradient Descent (random direction naive)" if phiMode == "random"
                    else "Gradient Descent (fixed-angle naive)")

    metrics = {
        "method": method_label,
        "converged": converged,
        "stopReason": stopReason,
        "iterations": kstar,
        "finalX": best.copy(),
        "finalFx": fxs[-1],
        "gradNorm": norm(df(best), normOrder),
        "stepNorm": stepNorms[-1] if stepNorms else None,
        "approxError": errors[-1] if errors else norm(df(best), normOrder),
        "alpha": alpha,
        "timeSec": timeSec,
        "history": {
            "k": np.array(ks),
            "gradNorms": np.array(gradNorms),
            "stepNorms": np.array(stepNorms),
            "approxErrors": np.array(approxErrors),
            "angles": np.array(angles),
            "directions": np.array(dirs, dtype=float) if len(dirs) else None,
            "xs2D": np.array(xs2D) if xs2D else None,
        },
        "seed": randomState,
    }
    return best, np.array(xs), np.array(fxs), np.array(errors), metrics

In [ ]:
def gradientDescentRandom(
    f, df, x0,
    alpha=0.1, maxIter=500, tol=1e-4, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=SEED, verbose=False
):
    """
    Descenso gradiente naïve con dirección aleatoria.
    """
    extra = {"phiMode": "random"}  # se arma internamente
    return gradientDescentNaive(
        f, df, x0,
        alpha=alpha, maxIter=maxIter, tol=tol, stopCrit=stopCrit,
        normOrder=normOrder, isPlottable=isPlottable,
        randomState=randomState, verbose=verbose, extra=extra
    )

def steepestDescent(
    f, df, x0,
    alpha=0.1, maxIter=500, tol=1e-4, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=SEED, verbose=False
):
    """
    Descenso máximo (steepest) naïve — equivale a ángulo fijo 0
    """
    extra = {"phiMode": "fixed", "phi": 0.0}  # se arma internamente
    return gradientDescentNaive(
        f, df, x0,
        alpha=alpha, maxIter=maxIter, tol=tol, stopCrit=stopCrit,
        normOrder=normOrder, isPlottable=isPlottable,
        randomState=randomState, verbose=verbose, extra=extra
    )

## Descenso gradiente de Newton, con Hessiano exacto

In [ ]:
def newtonDescent(
    f, df, x0, alpha=1.0, maxIter=100, tol=1e-6, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=SEED, verbose=False, extra=None
):
    """
    Método de Newton con Hessiano exacto y paso constante alpha.
    Retorna (best, xs, fxs, errors, metrics).

    extra:
      - ddf: callable Hessiano H(x) o matriz fija (np.ndarray, n x n)
      - solveSystem: "solve" (default) | "inv"
    """
    # ------- Validaciones de 'extra' -------
    extra = extra or {}
    if "ddf" not in extra or extra["ddf"] is None:
        raise ValueError("newtonDescent requiere extra['ddf'] (Hessiano exacto o callable).")
    ddf = extra["ddf"]
    solveSystem = extra.get("solveSystem", "solve")
    if solveSystem not in ("solve", "inv"):
        raise ValueError("extra['solveSystem'] debe ser 'solve' o 'inv'.")

    # ------- Estado inicial -------
    x = np.array(x0, dtype=float).reshape(-1)
    n = x.size

    xs, fxs, errors = [x.copy()], [f(x)], []
    gradNorms, stepNorms, approxErrors, dirs, xs2D, ks = [], [], [], [], [], []
    t0 = time.time()

    g = df(x)
    gradNorms.append(norm(g, ord=normOrder))
    converged, stopReason = False, "maxIter"

    if verbose:
        print(f"[k=0] fx={fxs[-1]:.6e} | ||grad||={gradNorms[-1]:.6e} | x={x}")

    # ------- Bucle principal -------
    for k in range(1, maxIter + 1):
        # Hessiano exacto (callable o matriz fija)
        H = ddf(x) if callable(ddf) else np.array(ddf, dtype=float)

        # Dirección de Newton: resolver H d = -g (sin invertir si es posible)
        try:
            if solveSystem == "solve":
                d = -np.linalg.solve(H, g)
            else:  # "inv"
                Hinv = np.linalg.inv(H)
                d = -(Hinv @ g)
        except np.linalg.LinAlgError:
            # Fallback numérico: pseudo-inversa
            d = -(np.linalg.pinv(H) @ g)

        # Asegurar dirección de descenso si es posible (g^T d < 0)
        gd = float(g @ d)
        if not np.isfinite(gd) or gd >= 0:
            # Fallback a steepest descent si Newton no da descenso
            d = -g
            gd = float(g @ d)

        # Paso Naïve
        x_new = x + alpha * d
        fx_new = f(x_new)
        step = x_new - x

        # Error según stopCrit
        if stopCrit == "grad":
            err = norm(df(x_new), ord=normOrder)
        elif stopCrit == "fx":
            err = abs(fx_new - fxs[-1])
        elif stopCrit == "xAbs":
            err = norm(step, ord=normOrder)
        elif stopCrit == "xRel":
            err = norm(step, ord=normOrder) / max(1.0, norm(x_new, ord=normOrder))
        else:
            raise ValueError("stopCrit inválido (usa 'grad','fx','xAbs','xRel').")

        # Registro
        xs.append(x_new.copy()); fxs.append(fx_new); errors.append(err)
        gradNorms.append(norm(df(x_new), ord=normOrder))
        stepNorms.append(norm(step, ord=normOrder))
        approxErrors.append(err); dirs.append(d.copy()); ks.append(k)
        if isPlottable and n == 2:
            xs2D.append(x_new.copy())

        if verbose:
            print(f"[k={k}] fx={fx_new:.6e} | err({stopCrit})={err:.6e} | "
                  f"||grad||={gradNorms[-1]:.6e} | ||step||={stepNorms[-1]:.6e} | gTd={gd:.3e}")

        # Paro por tolerancia
        if err <= tol:
            converged, stopReason = True, "tolerance"
            x = x_new
            break

        # Avanzar
        x, g = x_new, df(x_new)

    # ------- Cierre -------
    timeSec = time.time() - t0
    best = x
    kstar = len(xs) - 1

    if verbose:
        print(f"==> stopReason={('tolerance' if converged else 'maxIter')} | "
              f"iters={kstar} | fx*={fxs[-1]:.6e} | ||grad*||={norm(df(best), ord=normOrder):.6e}")

    metrics = {
        "method": "Newton (exact Hessian, naive step)",
        "converged": converged,
        "stopReason": stopReason,
        "iterations": kstar,
        "finalX": best.copy(),
        "finalFx": fxs[-1],
        "gradNorm": norm(df(best), ord=normOrder),
        "stepNorm": stepNorms[-1] if stepNorms else None,
        "approxError": errors[-1] if errors else norm(df(best), ord=normOrder),
        "alpha": alpha,
        "timeSec": timeSec,
        "history": {
            "k": np.array(ks),
            "gradNorms": np.array(gradNorms),
            "stepNorms": np.array(stepNorms),
            "approxErrors": np.array(approxErrors),
            "angles": None,
            "directions": np.array(dirs, dtype=float) if len(dirs) else None,
            "xs2D": np.array(xs2D) if xs2D else None,
        },
        "seed": randomState,
        "solveSystem": solveSystem,
    }
    return best, np.array(xs), np.array(fxs), np.array(errors), metrics

## Gradiente conjugado (FR, HS o PR)

In [ ]:
def conjugateGradientDescent(
    f, df, x0, alpha=0.1, maxIter=500, tol=1e-4, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=SEED, verbose=False, extra=None
):
    """
    Gradiente Conjugado (no lineal) con paso constante (naïve).

    extra:
      - betaRule: "FR" (default) | "PR" | "PR+" | "HS"
      - restartEvery: int o None (reinicio periódico a -gradiente)
      - denomEps: flotante para estabilizar divisiones (default 1e-15)
      - ensureDescent: bool (default True) -> si g^T d>=0, reinicia a -g
    """
    extra = extra or {}
    betaRule = extra.get("betaRule", "FR").upper()
    restartEvery = extra.get("restartEvery", None)
    denomEps = float(extra.get("denomEps", 1e-15))
    ensureDescent = bool(extra.get("ensureDescent", True))

    x = np.array(x0, dtype=float).reshape(-1)
    n = x.size

    xs, fxs, errors = [x.copy()], [f(x)], []
    gradNorms, stepNorms, approxErrors, dirs, betas, xs2D, ks = [], [], [], [], [], [], []
    t0 = time.time()

    g = df(x)
    d = -g.copy()
    gradNorms.append(norm(g, normOrder))
    converged, stopReason = False, "maxIter"
    restartCount = 0

    if verbose:
        print(f"[k=0] fx={fxs[-1]:.6e} | ||grad||={gradNorms[-1]:.6e} | x={x} | rule={betaRule}")

    for k in range(1, maxIter + 1):
        # --- Paso naïve ---
        x_new = x + alpha * d
        fx_new = f(x_new)
        step = x_new - x

        # --- Error según stopCrit ---
        if stopCrit == "grad":
            err = norm(df(x_new), normOrder)
        elif stopCrit == "fx":
            err = abs(fx_new - fxs[-1])
        elif stopCrit == "xAbs":
            err = norm(step, normOrder)
        elif stopCrit == "xRel":
            err = norm(step, normOrder) / max(1.0, norm(x_new, normOrder))
        else:
            raise ValueError("stopCrit inválido (usa 'grad','fx','xAbs','xRel').")

        # --- Registro ---
        xs.append(x_new.copy()); fxs.append(fx_new); errors.append(err)
        gradNorms.append(norm(df(x_new), normOrder))
        stepNorms.append(norm(step, normOrder))
        approxErrors.append(err); dirs.append(d.copy()); ks.append(k)
        if isPlottable and n == 2: xs2D.append(x_new.copy())
        if verbose:
            print(f"[k={k}] fx={fx_new:.6e} | err({stopCrit})={err:.6e} | "
                  f"||grad||={gradNorms[-1]:.6e} | ||step||={stepNorms[-1]:.6e}")

        # --- Paro por tolerancia ---
        if err <= tol:
            converged, stopReason = True, "tolerance"
            x = x_new
            break

        # --- Avanzar ---
        g_new = df(x_new)
        y = g_new - g

        # --- β según regla ---
        gg = max(denomEps, float(g @ g))
        if betaRule == "FR":
            beta = float(g_new @ g_new) / gg
        elif betaRule in ("PR", "PR+"):
            beta_raw = float(g_new @ (g_new - g)) / gg
            beta = max(0.0, beta_raw) if betaRule == "PR+" else beta_raw
        elif betaRule == "HS":
            denom = float(d @ y)
            beta = (float(g_new @ y) / (denom if abs(denom) > denomEps else np.sign(denom)*denomEps))
        else:
            raise ValueError("extra['betaRule'] debe ser 'FR'|'PR'|'PR+'|'HS'.")

        # --- Posible reinicio periódico ---
        if isinstance(restartEvery, int) and restartEvery > 0 and (k % restartEvery == 0):
            beta = 0.0
            restartCount += 1

        # --- Nueva dirección ---
        d_new = -g_new + beta * d

        # --- Asegurar descenso si se solicita ---
        if ensureDescent:
            gtd = float(g_new @ d_new)
            if (not np.isfinite(gtd)) or (gtd >= 0):
                d_new = -g_new
                restartCount += 1

        betas.append(beta)

        # Actualizar estado
        x, g, d = x_new, g_new, d_new

    # --- Cierre ---
    timeSec = time.time() - t0
    best = x
    kstar = len(xs) - 1

    if verbose:
        print(f"==> stopReason={('tolerance' if converged else 'maxIter')} | "
              f"iters={kstar} | fx*={fxs[-1]:.6e} | ||grad*||={norm(df(best), normOrder):.6e} | "
              f"restarts={restartCount}")

    method_label = f"Nonlinear Conjugate Gradient (naive, {betaRule})"
    metrics = {
        "method": method_label,
        "converged": converged,
        "stopReason": stopReason,
        "iterations": kstar,
        "finalX": best.copy(),
        "finalFx": fxs[-1],
        "gradNorm": norm(df(best), normOrder),
        "stepNorm": stepNorms[-1] if stepNorms else None,
        "approxError": errors[-1] if errors else norm(df(best), normOrder),
        "alpha": alpha,
        "timeSec": timeSec,
        "history": {
            "k": np.array(ks),
            "gradNorms": np.array(gradNorms),
            "stepNorms": np.array(stepNorms),
            "approxErrors": np.array(approxErrors),
            "angles": None,
            "directions": np.array(dirs, dtype=float) if len(dirs) else None,
            "betas": np.array(betas) if betas else None,
            "xs2D": np.array(xs2D) if xs2D else None,
        },
        "seed": randomState,
        "betaRule": betaRule,
        "restartEvery": restartEvery,
        "restarts": restartCount,
        "ensureDescent": ensureDescent,
    }
    return best, np.array(xs), np.array(fxs), np.array(errors), metrics

## Método BFGS

In [ ]:
def bfgsDescent(
    f, df, x0, alpha=0.1, maxIter=500, tol=1e-6, stopCrit="grad",
    normOrder=2, isPlottable=True, randomState=None, verbose=False, extra=None
):
    """
    BFGS (quasi-Newton) con paso constante (naive).

    extra:
      - H0:        np.ndarray (n x n) inversa-Hessiana inicial (default: I)
      - skipUpdateIf: float (default: 1e-12)  # umbral para y^T s; si <=, saltar actualización
      - ensureDescent: bool (default: True)   # si g^T d >= 0, forzar d = -g
    """
    extra = extra or {}
    skipUpdateIf  = float(extra.get("skipUpdateIf", 1e-12))
    ensureDescent = bool(extra.get("ensureDescent", True))

    # --- Estado inicial ---
    x = np.array(x0, dtype=float).reshape(-1)
    n = x.size

    # H0
    if "H0" in extra and extra["H0"] is not None:
        H = np.array(extra["H0"], dtype=float)
        if H.shape != (n, n):
            raise ValueError("extra['H0'] debe tener shape (n,n).")
    else:
        H = np.eye(n, dtype=float)

    xs, fxs, errors = [x.copy()], [f(x)], []
    gradNorms, stepNorms, approxErrors, dirs, xs2D, ks = [], [], [], [], [], []
    t0 = time.time()

    g = df(x)
    gradNorms.append(norm(g, normOrder))
    converged, stopReason = False, "maxIter"

    if verbose:
        print(f"[k=0] fx={fxs[-1]:.6e} | ||grad||={gradNorms[-1]:.6e} | x={x}")

    # --- Bucle principal ---
    for k in range(1, maxIter + 1):
        # Dirección BFGS
        d = -(H @ g)

        if ensureDescent:
            gtd = float(g @ d)
            if (not np.isfinite(gtd)) or (gtd >= 0):
                d = -g  # respaldo

        # Paso naïve
        x_new = x + alpha * d
        fx_new = f(x_new)
        step = x_new - x

        # Error según stopCrit
        if stopCrit == "grad":
            err = norm(df(x_new), normOrder)
        elif stopCrit == "fx":
            err = abs(fx_new - fxs[-1])
        elif stopCrit == "xAbs":
            err = norm(step, normOrder)
        elif stopCrit == "xRel":
            err = norm(step, normOrder) / max(1.0, norm(x_new, normOrder))
        else:
            raise ValueError("stopCrit inválido (usa 'grad','fx','xAbs','xRel').")

        # Registro
        xs.append(x_new.copy()); fxs.append(fx_new); errors.append(err)
        gradNorms.append(norm(df(x_new), normOrder))
        stepNorms.append(norm(step, normOrder))
        approxErrors.append(err); dirs.append(d.copy()); ks.append(k)
        if isPlottable and n == 2:
            xs2D.append(x_new.copy())

        if verbose:
            print(f"[k={k}] fx={fx_new:.6e} | err({stopCrit})={err:.6e} | "
                  f"||grad||={gradNorms[-1]:.6e} | ||step||={stepNorms[-1]:.6e}")

        # Paro por tolerancia
        if err <= tol:
            converged, stopReason = True, "tolerance"
            x = x_new
            break

        # Preparar actualización BFGS
        g_new = df(x_new)
        s = x_new - x
        y = g_new - g
        yTs = float(y @ s)

        # Actualización (si hay curvatura suficiente)
        if np.isfinite(yTs) and (yTs > skipUpdateIf):
            rho = 1.0 / yTs
            I = np.eye(n)
            # (I - rho s y^T) H (I - rho y s^T) + rho s s^T
            V = I - rho * np.outer(s, y)
            H = V @ H @ V.T + rho * np.outer(s, s)
            # Simetrizar por estabilidad numérica
            H = 0.5 * (H + H.T)
        # si no, se mantiene H

        # Avanzar
        x, g = x_new, g_new

    # --- Cierre ---
    timeSec = time.time() - t0
    best = x
    kstar = len(xs) - 1

    if verbose:
        print(f"==> stopReason={('tolerance' if converged else 'maxIter')} | "
              f"iters={kstar} | fx*={fxs[-1]:.6e} | ||grad*||={norm(df(best), normOrder):.6e}")

    metrics = {
        "method": "BFGS (naive step)",
        "converged": converged,
        "stopReason": stopReason,
        "iterations": kstar,
        "finalX": best.copy(),
        "finalFx": fxs[-1],
        "gradNorm": norm(df(best), normOrder),
        "stepNorm": stepNorms[-1] if stepNorms else None,
        "approxError": errors[-1] if errors else norm(df(best), normOrder),
        "alpha": alpha,
        "timeSec": timeSec,
        "history": {
            "k": np.array(ks),
            "gradNorms": np.array(gradNorms),
            "stepNorms": np.array(stepNorms),
            "approxErrors": np.array(approxErrors),
            "angles": None,
            "directions": np.array(dirs, dtype=float) if len(dirs) else None,
            "xs2D": np.array(xs2D) if xs2D else None,
        },
        "seed": randomState,
        "skipUpdateIf": skipUpdateIf,
        "ensureDescent": ensureDescent,
    }
    return best, np.array(xs), np.array(fxs), np.array(errors), metrics

# Problema 2

- Testar sus algoritmos del Ejercicio 1.
- Para las funciones 2D, muestre visualizaciones de la secuencia de aproximaciones $\{x_k\}$ convergiendo al mínimo local de su función.

  ![Ejemplo Gráfica](../images/example_graph.png)

- Elabore gráficas que muestren el error de aproximación, en función del número de iteración, y muestre la comparación de la evolución de la convergencia en sus cinco métodos. A partir de estas gráficas, discuta cuál de los métodos es más efectivo, en cada caso. Para ello, debe tomar en cuenta:
  - La solución aproximada obtenida
  - El error de aproximación
  - La norma del gradiente en la solución

- En cada uno de los casos, hallar un tamaño de paso $\alpha$ que garantice la convergencia de los métodos, y elabore una tabla comparativa de los resultados, error, número de iteraciones requeridas por cada método. Por ejemplo:

  | Algoritmo de optimización    | Convergencia (Sí/No) | Número de Iteraciones | Solución | Error |
  | ---------------------------- | -------------------- | --------------------- | -------- | ----- |
  | Descenso gradiente           |                      |                       |          |       |
  | Descenso gradiente aleatorio |                      |                       |          |       |
  | Descenso máximo              |                      |                       |          |       |
  | Descenso de Newton           |                      |                       |          |       |
  | Fletcher-Reeves              |                      |                       |          |       |
  | BFGS                         |                      |                       |          |       |

## Inciso a

La función $f : \mathbb{R}^2 \to \mathbb{R}$, dada por

$$
f(x, y) = x^4 + y^4 - 4xy + \frac{1}{2}y + 1.
$$

Punto inicial: $x_0 = (-3, 1, -3, 1)^T$, Óptimo: $x^* = (-1.01463, -1.04453)^T, \; f(x^*) = -1.51132$.

## Inciso b

La función de Rosembrock 2-dimensional $f : \mathbb{R}^2 \to \mathbb{R}$, dada por

$$
f(x_1, x_2) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2.
$$

Punto inicial: $x_0 = (-1.2, 1)^T$, Óptimo: $x^* = (1, 1)^T, \; f(x^*) = 0$.

## Inciso c

La función de Rosembrock 7-dimensional $f : \mathbb{R}^7 \to \mathbb{R}$, dada por

$$
f(x) = \sum_{i=1}^6 100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2.
$$

Punto inicial: $x_0 = (-1.2, 1, 1, 1, 1, -1.2, 1)^T$, Óptimo: $x^* = (1, 1, \dots, 1)^T, \; f(x^*) = 0$.